In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchinfo import summary

In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
torch.__version__

'2.8.0'

In [4]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        # First convolutional layer
        self.conv_layer = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3, 3))
        self.relu1 = nn.ReLU()
        # Max pooling layer
        self.maxpool = nn.MaxPool2d(kernel_size=(2, 2))

        # Calculate the size of the flattened features after conv and pooling
        # Input: (3, 200, 200)
        # After conv_layer: (32, 200-3+1, 200-3+1) = (32, 198, 198)
        # After maxpool: (32, 198/2, 198/2) = (32, 99, 99)
        self.flatten_features = 32 * 99 * 99  # 313632

        # Linear layer with 64 neurons
        self.fc1 = nn.Linear(self.flatten_features, 64)
        self.relu2 = nn.ReLU()

        # Output layer with 1 neuron. Sigmoid will be applied by the loss function for numerical stability.
        self.fc2 = nn.Linear(64, 1)
        # self.sigmoid = nn.Sigmoid() # Removed as BCEWithLogitsLoss handles it internally

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.relu1(x)
        x = self.maxpool(x)
        x = x.view(-1, self.flatten_features)  # Flatten the tensor
        x = self.fc1(x)
        x = self.relu2(x)
        x = self.fc2(x)
        # x = self.sigmoid(x) # Removed as BCEWithLogitsLoss handles it internally
        return x

In [5]:
# Instantiate the model
model = SimpleCNN()

# Define the optimizer
optimizer = optim.SGD(model.parameters(), lr=0.002, momentum=0.8)

# Define the loss function
criterion = nn.BCEWithLogitsLoss()

print(
    "Model and optimizer have been successfully defined. Sigmoid removed from model, criterion is BCEWithLogitsLoss."
)

Model and optimizer have been successfully defined. Sigmoid removed from model, criterion is BCEWithLogitsLoss.


# Question 2

What's the total number of parameters of the model? You can use torchsummary or count manually.

In [6]:
# Print the model summary
summary(model, input_size=(1, 3, 200, 200))

Layer (type:depth-idx)                   Output Shape              Param #
SimpleCNN                                [1, 1]                    --
├─Conv2d: 1-1                            [1, 32, 198, 198]         896
├─ReLU: 1-2                              [1, 32, 198, 198]         --
├─MaxPool2d: 1-3                         [1, 32, 99, 99]           --
├─Linear: 1-4                            [1, 64]                   20,072,512
├─ReLU: 1-5                              [1, 64]                   --
├─Linear: 1-6                            [1, 1]                    65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 55.20
Input size (MB): 0.48
Forward/backward pass size (MB): 10.04
Params size (MB): 80.29
Estimated Total Size (MB): 90.81

Total params: 20,073,473

# Generators and Training


In [7]:
import torchvision.transforms as transforms

train_transforms = transforms.Compose(
    [
        transforms.Resize((200, 200)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]
        ),  # ImageNet normalization
    ]
)

In [8]:
import torchvision.datasets as datasets
from torch.utils.data import DataLoader

# Define device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define datasets
train_dataset = datasets.ImageFolder("data/train", transform=train_transforms)
validation_dataset = datasets.ImageFolder("data/test", transform=train_transforms)

# Define data loaders
batch_size = 20
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)

num_epochs = 10
history = {"acc": [], "loss": [], "val_acc": [], "val_loss": []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(
            1
        )  # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history["loss"].append(epoch_loss)
    history["acc"].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history["val_loss"].append(val_epoch_loss)
    history["val_acc"].append(val_epoch_acc)

    print(
        f"Epoch {epoch + 1}/{num_epochs}, "
        f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
        f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}"
    )

Epoch 1/10, Loss: 0.6325, Acc: 0.6462, Val Loss: 0.6152, Val Acc: 0.6318
Epoch 2/10, Loss: 0.5322, Acc: 0.7200, Val Loss: 0.6898, Val Acc: 0.6517
Epoch 3/10, Loss: 0.4877, Acc: 0.7488, Val Loss: 0.6210, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4419, Acc: 0.7925, Val Loss: 0.7786, Val Acc: 0.6219
Epoch 5/10, Loss: 0.3772, Acc: 0.8275, Val Loss: 0.6921, Val Acc: 0.6517
Epoch 6/10, Loss: 0.3170, Acc: 0.8775, Val Loss: 0.6288, Val Acc: 0.6667
Epoch 7/10, Loss: 0.2242, Acc: 0.9213, Val Loss: 0.7828, Val Acc: 0.6965
Epoch 8/10, Loss: 0.2917, Acc: 0.8912, Val Loss: 0.7796, Val Acc: 0.6965
Epoch 9/10, Loss: 0.1384, Acc: 0.9550, Val Loss: 0.8441, Val Acc: 0.7114
Epoch 10/10, Loss: 0.1230, Acc: 0.9600, Val Loss: 0.7801, Val Acc: 0.7114


# Question 3

What is the median of training accuracy for all the epochs for this model?

In [9]:
np.median(history['acc'])

np.float64(0.8525)

Median of training accuracy for all epochs is 0.852

# Question 4

What is the standard deviation of training loss for all the epochs for this model?

In [10]:
np.std(history['loss'])

np.float64(0.16041681503637747)

Standard deviation for training loss for all epochs is 0.16

# Data Augmentation

In [11]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [ ]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.0606, Acc: 0.9912, Val Loss: 0.9458, Val Acc: 0.7065
Epoch 2/10, Loss: 0.0887, Acc: 0.9675, Val Loss: 0.9489, Val Acc: 0.7413
Epoch 3/10, Loss: 0.0280, Acc: 0.9962, Val Loss: 1.0687, Val Acc: 0.7065
Epoch 4/10, Loss: 0.0143, Acc: 1.0000, Val Loss: 1.0184, Val Acc: 0.7413
Epoch 5/10, Loss: 0.0101, Acc: 1.0000, Val Loss: 1.1018, Val Acc: 0.7413
Epoch 6/10, Loss: 0.0078, Acc: 1.0000, Val Loss: 1.1314, Val Acc: 0.7313
Epoch 7/10, Loss: 0.0062, Acc: 1.0000, Val Loss: 1.2250, Val Acc: 0.7264
Epoch 8/10, Loss: 0.0050, Acc: 1.0000, Val Loss: 1.2015, Val Acc: 0.7313
Epoch 9/10, Loss: 0.0041, Acc: 1.0000, Val Loss: 1.2306, Val Acc: 0.7313
Epoch 10/10, Loss: 0.0036, Acc: 1.0000, Val Loss: 1.2536, Val Acc: 0.7313


# Question 5

What is the mean of test loss for all the epochs for the model trained with augmentations?

In [13]:
np.mean(history['val_loss'])

np.float64(1.1125859447235973)

# Question 6

What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

In [16]:
np.mean(history['val_acc'][5:])

np.float64(0.7303482587064678)

0.73